# Event by Event Data Exploration
In event by event data explaration we load the data into a TChain and can then get individual events, or loop over events. To do this, you do *not* have to load the MiniDST library, however doing so really simplifies access to the data, especially in Python. Alternatively you can use TChain to explore the data with the Draw() method. This works better at the ROOT prompt than in a notebook. 
For more sophisticated data analysis, I woud recommend using RDataFrame to analyze the data. This is covered in a separate notebook.

## Loading data
We first setup ROOT and load the MiniDST library. We next create a TChain to open the ROOT files and connect this to a MiniDST class object.

In [2]:
import numpy as np
import ROOT as R
# R.EnableImplicitMT()
# %jsroot on
R.gSystem.Load("/data/HPS/lib/libMiniDST.dylib")   # Load the library. The path will differ on your own machine.
ch = R.TChain("MiniDST")                           # Setup a TChain.
ch.Add("/data/HPS/data/physrun2021/Tritrig/Pure/HPS_Run2021Pass1_v5_1pt92GeV_x0_minidst.root") # Load data
mdst = R.MiniDst()                                 # Create a MiniDST class object
mdst.use_mc_particles = True
mdst.DefineBranchMap()                             # Define the branches in the class.
# mdst.use_ecal_cluster_uncor = True
mdst.SetBranchAddressesOnTree(ch)                  # Connect the TChain with the data to the MiniDST class



In [3]:
def print_daughters(mdst, i_part, indent=0):
    """Given an index to an MC particle, print all the daughters.
    This function will recursively print the daughters of the daughters.
    Arguments:
        mdst   -- a MiniDst object that was linked to a TTree
        i_part -- the index of the particle to print
        ident  -- the amount of indentation of the output.
    """

    if mdst.mc_part_pdg.size() > 0:
        part_pdg = mdst.mc_part_pdg[i_part]
    else:
        part_pdg = 0
    print(" "*indent+f" {i_part:3d}  pdg: {part_pdg:4d}  E: {mdst.mc_part_energy[i_part]:9.6f} " +
          f"p = ({mdst.mc_part_px[i_part]:9.6f},{mdst.mc_part_py[i_part]:9.6f},{mdst.mc_part_pz[i_part]:9.6f})" +
          f"v = ({mdst.mc_part_x[i_part]:5.2f},{mdst.mc_part_y[i_part]:5.2f},{mdst.mc_part_z[i_part]:5.2f}) " +
          f"end=({mdst.mc_part_end_x[i_part]:5.2f},{mdst.mc_part_end_y[i_part]:5.2f},{mdst.mc_part_end_z[i_part]:5.2f})")
    if len(mdst.mc_part_daughters[i_part]) > 0:
        print(" "*(indent+14) + "| ")
        for i in range(len(mdst.mc_part_daughters[i_part])):
            ii = mdst.mc_part_daughters[i_part][i]  # Get the daughter reference
            print_daughters(mdst, ii, indent+11)            # Print by recursing


def print_mc_particle_tree(mdst):
    """Print the MCParticle tree.
    Arguments:
        mdst -- a MiniDst object that was linked to a TTree.
    """
    for i in range(len(mdst.mc_part_parents)):
        if len(mdst.mc_part_parents[i]) == 0:  # top level particle
            print_daughters(mdst, i, 0)


In [12]:
ch.GetEvent(105)

11906

In [13]:
print(f"Run number: {mdst.run_number:5d}, Event number: {mdst.event_number:8d}  trigger: 0x{mdst.trigger:04X}")
print(f"Ecal hits         (ix,iy): {list(zip(mdst.ecal_hit_index_x, mdst.ecal_hit_index_y))}")
print(f"Ecal cluster seed (ix,iy): {list(zip(mdst.ecal_cluster_seed_ix,mdst.ecal_cluster_seed_iy))}")
print(f"List Ecal cluster      (x,y,z): ",end="")
for i in range(len(mdst.ecal_cluster_x)):
    print(f"({mdst.ecal_cluster_x[i]:5.1f},{mdst.ecal_cluster_y[i]:5.1f},{mdst.ecal_cluster_z[i]:5.1f})",end=", ")
print()
print(f"List Ecal cluster E : {mdst.ecal_cluster_energy}")

Run number: 14700, Event number:     5328  trigger: 0x0000
Ecal hits         (ix,iy): [(17, -2), (17, -3), (17, -3), (15, -2), (18, -3), (23, -4), (17, -1), (16, -1), (16, -2), (-19, -4)]
Ecal cluster seed (ix,iy): [(17, -2)]
List Ecal cluster      (x,y,z): (264.8,-40.8,1450.1), 
List Ecal cluster E : { 1.3979999 }


In [10]:
ievt=1

In [13]:
go=True
while go:
    ch.GetEvent(ievt)
    ievt += 1
    top_parent=-1
    for i in range(len(mdst.mc_part_pdg)):
        if mdst.mc_part_pdg[i]==623:
            top_parent=i
    n_ele=0
    n_pos=0
    for idx in mdst.mc_part_daughters[top_parent]:
        if mdst.mc_part_pdg[idx] == 11:
            n_ele+=1
        if mdst.mc_part_pdg[idx] == -11:
            n_pos+=1
    if n_ele == 1:
        print(f"{ievt}: Run number: {mdst.run_number:5d}, Event number: {mdst.event_number:8d}  trigger: 0x{mdst.trigger:04X}")
        print_mc_particle_tree(mdst)
        go=False


38045: Run number: 14700, Event number:      715  trigger: 0x0000
   8  pdg:  623  E:  0.100000 p = ( 0.000000, 0.000000, 0.000000)v = ( 0.04, 0.00, 0.90) end=( 0.04, 0.00, 0.92)
              | 
              1  pdg:  -11  E:  1.229974 p = ( 0.063907, 0.020928, 1.228135)v = ( 0.04, 0.00, 0.92) end=(285.32,30.13,1455.60)
                         | 
                         0  pdg:   22  E:  0.000000 p = ( 0.000000, 0.000000, 0.000000)v = (264.63,25.04,1385.25) end=(284.27,26.69,1456.40)
                                    | 
                                    5  pdg:  -11  E:  0.000511 p = ( 0.000000, 0.000000,-0.000000)v = (284.27,26.69,1456.40) end=(286.10,27.57,1468.84)
                                               | 
                                               6  pdg:   22  E:  0.000000 p = ( 0.000000, 0.000000, 0.000000)v = (286.51,26.93,1466.12) end=(286.70,27.00,1467.61)
                                                          | 
                                           

In [75]:
for ievt in range(120,400):
    ch.GetEvent(ievt)
    top_parent=-1
    for i in range(len(mdst.mc_part_pdg)):
        if mdst.mc_part_pdg[i]==623:
            top_parent=i        
    if mdst.mc_part_daughters[top_parent].size()>3:
        for j in mdst.mc_part_daughters[top_parent]:
            print(f"{mdst.mc_part_pdg[j]}",end=", ")
        print()

11, -11, 11, 22, 
11, -11, 11, 22, 
11, 11, 22, -11, 
11, 22, 11, -11, 
11, 11, 22, -11, 
11, 11, 22, 22, -11, 
11, -11, 11, 22, 
22, -11, 11, 11, 11, 
11, 22, -11, 11, 
22, -11, 11, 11, 
11, 22, 11, -11, 22, 
-11, 22, 11, 11, 
11, 22, -11, 11, 
22, 11, -11, 11, 
22, -11, 11, 11, 
11, 22, 11, -11, 
-11, 11, 22, 11, 
11, -11, 22, 11, 
-11, 22, 11, 11, 
11, -11, 22, 11, 
22, 11, 11, -11, 
-11, 11, 22, 11, 
-11, 11, 11, 11, 
-11, 11, 11, 11, 
11, -11, 11, 11, 
11, 11, 22, -11, 
11, -11, 22, 11, 
-11, 11, 11, 22, 
11, 11, 22, -11, 
11, -11, 11, 22, 
-11, 11, 22, 11, 
11, 11, 22, -11, 
11, -11, 11, 22, 
-11, 11, 11, 22, 
11, 11, 11, -11, 
-11, 11, 11, 22, 
22, 11, -11, 11, 
-11, 11, 22, 11, 
22, -11, 11, 11, 
22, -11, 11, 11, 
11, -11, 11, 22, 


In [52]:
print_mc_particle_tree(mdst)

   8  pdg:  623  E:  0.100000 p = ( 0.000000, 0.000000, 0.000000)v = ( 0.16, 0.00, 0.90) end=( 0.16, 0.00, 0.92)
              | 
             13  pdg:   11  E:  2.138439 p = ( 0.029547,-0.047549, 2.137706)v = ( 0.16, 0.00, 0.92) end=(-204.32,-54.83,2500.00)
             14  pdg:   22  E:  0.069725 p = ( 0.003875, 0.002417, 0.069576)v = ( 0.16, 0.00, 0.92) end=(84.02,52.31,1506.69)
             19  pdg:   11  E:  0.413947 p = ( 0.022363, 0.014619, 0.413084)v = ( 0.16, 0.00, 0.92) end=(-1089.58,84.00,2500.00)
                         | 
                         0  pdg:   22  E:  0.003408 p = (-0.001616, 0.000135, 0.002997)v = (-187.04,31.08,845.77) end=(-111.13,-149.98,1215.20)
                                    | 
                                   17  pdg:   11  E:  0.001647 p = (-0.001012,-0.000168, 0.001183)v = (-199.23,27.81,883.03) end=(-199.38,27.78,883.20)
                         2  pdg:   22  E:  0.003908 p = (-0.001758, 0.000328, 0.003475)v = (-193.77,31.98,858.99) end=(-250